In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer
from amrlib.models.parse_spring.tokenization_bart import AMRBartTokenizer
from transformers import AutoConfig

from utils import *

/home/david/.local/share/virtualenvs/SemanticsNLPProject-2N7QzmHs/lib/python3.9/site-packages/transformers/models/bart/modeling_bart.py:527: FutureWarning: The class `PretrainedBartModel` has been depreciated, please use `BartPretrainedModel` instead.
  warnings.warn(


In [2]:
#copied from https://github.com/muyeby/AMRBART/blob/5461144b2e1984d324919f459028b46b335d4496/spring/spring_amr/utils.py

model, tokenizer = instantiate_model_and_tokenizer()


#

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'AMRBartTokenizer'.


AttributeError: 'BartConfig' object has no attribute 'static_position_embeddings'

In [26]:
example_english_phrase = "ETH requires hard work."
batch = tokenizer(example_english_phrase, return_tensors="pt")
generated_ids = model.generate(batch["input_ids"])


tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

TypeError: sequence item 8: expected str instance, NoneType found